# Social Media APIs

## Getting data from the twitter API

### BEFORE YOU START

You need to "pip install twitter" before you can use this

You also need to get 'access tokens' before you can use the Twitter API. Here's how:

* go to https://apps.twitter.com/
* click on "create new app".  Fill in some details (the actual web address doesn't matter; I often use my own website address http://www.overcognition.com or www.example.com when I'm testing; you also don't have to put anything in the "callback URL" box)
* Agree to the developer agreement (click the tickbox), then click "create your app"
* You're halfway there.  Click on the "Keys and Access Tokens" tab at the top of the page
* Click on "create my access token" at the bottom of this screen

You now have everything you need on this page. Here's what's what: 
* OAUTH_TOKEN: Access token (bottom of the page)
* OAUTH_TOKEN_SECRET: Access token (bottom of the page)
* CONSUMER_KEY: Consumer key (top of the page)
* CONSUMER_SECRET: Consumer secret (top of the page)

Copy these, very carefully, into a file, with commas between them, e.g. 'OAUTH_TOKEN,OAUTH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET'

### First, set up your link to the Twitter API

You need to set these 4 parameters to your Twitter API keys: OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET. Best practice is to have a file somewhere else in your system (e.g. outside anything going into a github repository) called something like "twitter secrets" and read them in from there, so you don't accidentally add your private keys to e.g. a github repository.

We're using Twitter's REST API here (https://dev.twitter.com/rest/public): you request a dataset (e.g. Fema's recent tweets), Twitter sends you back that dataset.  If you want to monitor tweets in real-time, you'll have to use Twitter's streaming API instead.  More details on the streaming API here: http://socialmedia-class.org/twittertutorial.html,  https://dev.twitter.com/streaming/overview

In [10]:
import twitter

# Change this filename to the one that you've put your own Twitter keys into
# File has one line, with keys separated by commas:
# OAUTH_TOKEN,OAUTH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET
fsecret = open('../../../../twittersecrets.txt', 'r')
secrets = fsecret.readline()
ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET = secrets.strip().split(',')
ACCESS_TOKEN = ACCESS_TOKEN.strip()
ACCESS_TOKEN_SECRET = ACCESS_TOKEN_SECRET.strip()
CONSUMER_KEY = CONSUMER_KEY.strip()
CONSUMER_SECRET = CONSUMER_SECRET.strip()

auth = twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
ta = twitter.Twitter(auth=auth)

### Get tweets for a specific hashtag

In [21]:
tweetdata = ta.search.tweets(q='#migrantcrisis')
print('Returned {} tweets'.format(len(tweetdata['statuses'])))
print('* Tweet data keys: {}'.format(tweetdata.keys()))
print('* Information about your search: {}'.format(tweetdata['search_metadata']))
print('First Tweet:')
print('* First tweet\'s keys: {}'.format(tweetdata['statuses'][0].keys()))
print('* First tweet\'s data: {}'.format( tweetdata['statuses'][0]))

Returned 15 tweets
* Tweet data keys: dict_keys(['search_metadata', 'statuses'])
* Information about your search: {'count': 15, 'next_results': '?max_id=707705076723408896&q=%23migrantcrisis&include_entities=1', 'since_id': 0, 'max_id_str': '707713117594198016', 'completed_in': 0.058, 'refresh_url': '?since_id=707713117594198016&q=%23migrantcrisis&include_entities=1', 'max_id': 707713117594198016, 'query': '%23migrantcrisis', 'since_id_str': '0'}
First Tweet:
* First tweet's keys: dict_keys(['favorited', 'in_reply_to_user_id', 'id', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'created_at', 'entities', 'place', 'is_quote_status', 'contributors', 'user', 'source', 'in_reply_to_screen_name', 'lang', 'id_str', 'retweeted', 'text', 'retweet_count', 'coordinates', 'metadata', 'possibly_sensitive', 'favorite_count', 'truncated', 'in_reply_to_status_id', 'geo'])
* First tweet's data: {'favorited': False, 'in_reply_to_user_id': None, 'id': 707713117594198016, 'in_reply_to_status_id_

### Get tweets from a specific timeline

home_timeline is your own timeline

In [14]:
mytweets = ta.statuses.home_timeline()
fematweets = ta.statuses.user_timeline(screen_name="fema")

### Get tweets from a specific geographical area

A woeid is an identifier for a specific place. For example, 
World 1
USA 23424977
UK  23424975
Kenya 23424863
Tanzania 23424973 - not supported by twitter
Yemen 23425002 - not supported by twitter

See https://blog.twitter.com/2010/woeids-twitters-trends and 
https://developer.yahoo.com/geo/geoplanet/guide/concepts.html for more about woeids

Woeid lookup here: http://woeid.rosselliot.co.nz/lookup/tanzania
OR can do this from python using yweather: http://stackoverflow.com/questions/22927307/how-to-find-woeid-where-on-earth-id-of-a-country
Note about twitter availability: https://moebiuscurve.wordpress.com/2014/10/08/twitter-woeid-available-trends/
Question about which geolocations Twitter uses for this search: http://stackoverflow.com/questions/28944536/geolocation-information-for-twitter-api-trends-available

See https://dev.twitter.com/docs/api/1.1/get/trends/place for twitter api call
See https://dev.twitter.com/overview/api/places for call parameters

In [15]:
woeid = 23424863
place_trends = ta.trends.place(_id=woeid)

### Post messages to Twitter. Beware, this will really happen!

Statuses.update posts to your own twitter stream
direct_messages.new() sends a direct message to someone else

In [ ]:
# ta.statuses.update(status="I just posted from Python!")
# ta.direct_messages.new(user=myfriendsname, text=mytextmessage)

### Couple of functions that might be useful to you

In [22]:
import json

def write_tweets(tweetdata, outfile = 'example_data/mytweets.json'):
    fout = open(outfile, 'w')
    json.dump(tweetdata, fout)
    fout.close()
    return

In [30]:
import pandas as pd

rawresults = pd.DataFrame(tweetdata['statuses'])
print('{}'.format(rawresults['text']))
rawresults

0     Stirile zilei - 9 martie: https://t.co/R3n6Vr8...
1     https://t.co/I09bHPUEyM via @youtube @guardian...
2     RT @ECentauri: GERMAN GOVT. PROMOTES INTERRACI...
3     RT @vladokrstevski: #NeoCommies\n#UDBA\n#Soros...
4     RT @ECentauri: 'I GOT A SOLUTION TO #migrantcr...
5     #Migrants - Feeling Unwanted In Germany, Some ...
6     RT @INTHENOWRT: This drone captures some 14,00...
7     People who moan about the migrant crisis.... 🖕...
8     RT @London_360: .@RitaOra delivers a personal ...
9     RT @INTHENOWRT: This drone captures some 14,00...
10    @lksriv but on the otherhand ...\nhttps://t.co...
11    @zoharfisher @TheEconomist wonder if she think...
12    RT @newint: #PhotoGallery | #MigrantCrisis: As...
13    Germany/Japan should both accept ALL of the wo...
14    RT @petersthoughts: The Death of the Most Gene...
Name: text, dtype: object


,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Wed Mar 09 23:42:10 +0000 2016,{'media': [{'media_url_https': 'https://pbs.tw...,0,False,None,707713117594198016,707713117594198016,None,...,"{'iso_language_code': 'und', 'result_type': 'r...",None,False,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Stirile zilei - 9 martie: https://t.co/R3n6Vr8...,False,"{'verified': False, 'notifications': False, 'n..."
1,None,None,Wed Mar 09 23:41:45 +0000 2016,"{'hashtags': [{'indices': [47, 61], 'text': 'm...",0,False,None,707713013927772165,707713013927772165,None,...,"{'iso_language_code': 'und', 'result_type': 'r...",None,False,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",https://t.co/I09bHPUEyM via @youtube @guardian...,False,"{'verified': False, 'notifications': False, 'n..."
2,None,None,Wed Mar 09 23:41:29 +0000 2016,{'media': [{'media_url_https': 'https://pbs.tw...,0,False,None,707712946319667200,707712946319667200,None,...,"{'iso_language_code': 'en', 'result_type': 're...",None,False,22,False,"{'favorited': False, 'in_reply_to_user_id': No...","<a href=""http://twitter.com/download/android"" ...",RT @ECentauri: GERMAN GOVT. PROMOTES INTERRACI...,False,"{'verified': False, 'notifications': False, 'n..."
3,None,None,Wed Mar 09 23:40:21 +0000 2016,{'media': [{'media_url_https': 'https://pbs.tw...,0,False,None,707712659014213632,707712659014213632,None,...,"{'iso_language_code': 'und', 'result_type': 'r...",None,False,3,False,"{'favorited': False, 'in_reply_to_user_id': No...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @vladokrstevski: #NeoCommies\n#UDBA\n#Soros...,False,"{'verified': False, 'notifications': False, 'n..."
4,None,None,Wed Mar 09 23:35:52 +0000 2016,{'media': [{'media_url_https': 'https://pbs.tw...,0,False,None,707711532688248832,707711532688248832,None,...,"{'iso_language_code': 'en', 'result_type': 're...",None,False,37,False,"{'favorited': False, 'in_reply_to_user_id': No...","<a href=""http://twitter.com/#!/download/ipad"" ...",RT @ECentauri: 'I GOT A SOLUTION TO #migrantcr...,False,"{'verified': False, 'notifications': False, 'n..."
5,None,None,Wed Mar 09 23:22:29 +0000 2016,"{'hashtags': [{'indices': [0, 9], 'text': 'Mig...",0,False,None,707708162900496384,707708162900496384,None,...,"{'iso_language_code': 'en', 'result_type': 're...",None,False,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...","#Migrants - Feeling Unwanted In Germany, Some ...",False,"{'verified': False, 'notifications': False, 'n..."
6,None,None,Wed Mar 09 23:21:44 +0000 2016,"{'hashtags': [{'indices': [48, 57], 'text': 'r...",0,False,None,707707974383476737,707707974383476737,None,...,"{'iso_language_code': 'en', 'result_type': 're...",None,False,290,False,"{'favorited': False, 'in_reply_to_user_id': No...","<a href=""http://twitter.com/download/android"" ...","RT @INTHENOWRT: This drone captures some 14,00...",False,"{'verified': False, 'notifications': False, 'n..."
7,None,None,Wed Mar 09 23:21:29 +0000 2016,"{'hashtags': [{'indices': [48, 62], 'text': 'm...",0,False,None,707707913230483456,707707913230483456,None,...,"{'iso_language_code': 'en', 'result_type': 're...",None,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",People who moan about the migrant crisis.... 🖕...,False,"{'verified': False, 'notifications': False, 'n..."
8,None,None,Wed Mar 09 23:19:42 +0000 2016,{'media': [{'media_url_https': 'https://pbs.tw...,0,False,None,707707461369729024,707707461369729024,None,...,"{'iso_language_code': 'en', 'result_type': 're...",None,False,16,False,"{'favorited': False, 'in_reply_to_user_id': No...","<a href=""http://twitter.com/download/android"" ...",RT @London_360: .@RitaOra delivers a personal ...,False,"{'verified': False, 'notifications': False, 'n..."
9,None,None,Wed

## Getting data from the facebook API

In [ ]:
from facepy import GraphAPI
import json

#Get access token from https://developers.facebook.com/tools/explorer/
access = ‘your_facebook_access_token’
graph = GraphAPI(access)
page_id= ‘id_of_page_you’re_tracking’
datas= graph.get(page_id+'/posts?fields=message', page=True, retry=5)

In [ ]:
from facepy import GraphAPI 
access = ‘your_facebook_access_token’
graph = GraphAPI(access)
graph.get(‘me/posts')
graph.post(
        "me/feed",
        message="Posted this from Python. Data science class fascinated by ability to access social media from code...", 
        caption="Posted from Python", 
        description="posted from Python")

## Getting data from the Openweather API

In [7]:
import requests
import json
import csv

boundingbox = '29.256, -12.039, 40.561523, -0.922812'
apiurl = ('http://api.openweathermap.org/data/2.1/find/station?bbox=' + 
       boundingbox + ',10&cluster=no')
response = requests.get(url=apiurl)
jdata = json.loads(response.text)

fout = open('TZAweather.csv', 'w')
outcsv = csv.writer(fout)
outcsv.writerow(['name', 'latitude', 'longitude'])

for station in jdata['list']:
    sname = station['name']
    lat = station['coord']['lat']
    lon = station['coord']['lon']
    print("{} latlong: {},{}".format(sname, str(lat), str(lon)))
    outcsv.writerow([sname, lat, lon])
    
fout.close()

HRYR latlong: -1.9686,30.1395
KQVF latlong: -2.09,40.54
HKML latlong: -3.2293,40.1017
HKMO latlong: -4.0348,39.5943
HKJK latlong: -1.3192,36.9278
